### 설정

In [211]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

### 크롬 드라이버키기

In [202]:
driver = webdriver.Chrome('./chromedriver_90')

### url 접속

In [203]:
url = "https://www.amazon.com/"
driver.get(url)

### 버튼 및 검색창 찾기

In [204]:
sel_search = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
sel_btn = driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')

#print(sel_search, sel_btn)
print(sel_search.tag_name, sel_btn.tag_name)

input input


### 검색(click)

In [205]:
sel_search.clear()
sel_search.send_keys('wireless mouse')
sel_btn.click()

In [206]:
choice = driver.find_element_by_xpath('//*[@id="search"]/div[1]/div/div[1]/div/span[3]/div[2]/div[3]/div/span/div/div/div/div/div[2]/div[2]/div/div[1]/h2/a')
choice.click()

In [207]:
choice_1 = driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
choice_1.click()

### 제목확인

In [174]:
page = driver.page_source
soup = BeautifulSoup(page,'lxml')
soup.title

<title>Amazon.com: Customer reviews: Jelly Comb 2.4G Slim Wireless Mouse with Nano Receiver MS001 (Black and Gold)</title>

### 상품 이름

In [175]:
title = soup.find_all('span', class_ = 'a-size-large product-title-word-break')

for one in title:
    print(one.text.strip())

### 상품 리뷰

In [176]:
review = soup.find_all('div', class_ = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
for one in review:
    print(one.text.strip(), end = "\n\n\n\n")
len(review)

0

### 리뷰작성자

In [177]:
user = soup.find_all('span', class_ = 'a-profile-name')
for one in user:
    print(one.text)
len(user)

Sigrid Koo
Doc
Doc
Sigrid Koo
MonkeyPaw
AmazonCustomer
DJW
KM
chelizabeth
Lawren Freebody
Gabriel Martinez
Melissa Teller


12

### 평점

In [178]:
all_s = soup.find_all('div', class_ = 'a-section celwidget')
tmp = all_s[0].find('span', class_ = 'a-icon-alt').text
rating = re.findall('\d+', tmp)
rating[0]

'1'

In [179]:
all_score = []
for one in all_s:
    rating_one = one.find('span', class_ = 'a-icon-alt').text
    nums = re.findall('\d+', rating_one)[0]
    all_score.append(nums)
all_score

['1', '5', '5', '5', '5', '1', '1', '3', '5', '5']

### 날짜

In [180]:
date = soup.find_all('span', class_ = 'a-size-base a-color-secondary review-date')
for one in date:
    texts = one.text.split('on')
    print(texts[1])

 March 19, 2019
 September 22, 2017
 September 22, 2017
 March 19, 2019
 March 21, 2017
 October 5, 2017
 March 13, 2017
 September 16, 2017
 December 15, 2017
 July 26, 2017
 October 19, 2016
 March 17, 2020


# 여러페이지의 정보 가져오기

In [209]:
all_user = []    #사용자
all_reviews = []  #리뷰
all_ratings = [] #평점
all_dates = []    #날짜

for i in range(2,7,1):
    time.sleep(2)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    print(soup.title)
    
    all_adr = soup.find_all('div', class_ ='a-section celwidget')
    print(len(all_adr ))
    
    for one in all_adr:
        user_one = one.find('span', class_ = 'a-profile-name').text
        #print(user_one)
        all_user.append(user_one)
        
        rating_one = one.find('span', class_ = 'a-icon-alt').text
        
        nums = re.findall('\d+', rating_one)[0]
        #print(nums)
        all_ratings.append(nums)
        
        review_one = one.find('span', class_ = 'a-size-base review-text review-text-content')
        tmp = review_one.text
        review = tmp.strip()
        all_reviews.append(review)
        
        date_one = one.find('span', class_ = "a-size-base a-color-secondary review-date")
        texts = date_one.text.split('on')
        data = texts[1].strip()
        all_dates.append(data)
        
    print("user : ", all_user[-1], "rating : ", all_ratings[-1])
    print("review : ", all_reviews[-1], "date: ", end = "\n\n\n")
    
    
    
    sel_next = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    sel_next.click()

<title>Amazon.com: Customer reviews: Jelly Comb 2.4G Slim Wireless Mouse with Nano Receiver MS001 (Black and Gold)</title>
10
user :  Melissa Teller rating :  5
review :  Your browser does not support HTML5 video.


   I really like that it was easy to set up with my laptop.  The colors are great, and its very quiet.  I got  because my baby can hear a pin drop and she doesn't wake up when I click. it sometimes need an easy shake when in sleep mode but I don't think that's as issue. you'll know when it needs a new battery. it takes one AA battery. I didn't show the lil piece that plugs into the computer but its small and stays on the side of your computer in the USB outlet date: 


<title>Amazon.com: Customer reviews: Jelly Comb 2.4G Slim Wireless Mouse with Nano Receiver MS001 (Black and Gold)</title>
10
user :  Patti-Cakes from Dayton rating :  5
review :  Perfect for the budget-conscious, no-frills user.  I'm not a gamer.  I use my computer for work, email and surfing the web.  When 

## 데이터프레임으로 저장

In [216]:
total_rv= {'ratings' :all_ratings, 'reviews': all_reviews, 'user':all_user, 'date': all_dates}
total_rv = pd.DataFrame(total_rv)
total_rv

,ratings,reviews,user,date
0,1,Your browser does not support HTML5 video.\n\n...,Doc,"September 22, 2017"
1,5,Your browser does not support HTML5 video.\n\n...,Sigrid Koo,"March 19, 2019"
2,5,For the price this mouse is quite good. The to...,MonkeyPaw,"March 21, 2017"
3,5,Great wireless mouse. I love that it has an on...,AmazonCustomer,"October 5, 2017"
4,5,I don't really write that many reviews on Amaz...,DJW,"March 13, 2017"
5,1,Update 9/21/17: After initially leaving positi...,KM,"September 16, 2017"
6,1,"DO NOT BUY THIS MOUSE. i know it's cheap, i kn...",chelizabeth,"December 15, 2017"
7,3,"Lightweight and streamline, the click is barel...",Lawren Freebody,"July 26, 2017"
8,5,So far so good! I'll let you know if there is ...,Gabriel Martinez,"October 19, 2016"
9,5,Your browser does not support HTML5 video.\n\n...,Melissa Teller,"March 17, 2020"


## csv파일로 저장

In [218]:
total_rv.to_csv("Amazon_wirelessMouse_Review.csv", index = False)